In [ ]:
import pandas as pd
import glob
from pathlib import Path
import seaborn as sns
import os
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('avx-frequency-uncore-anomaly'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
levels = glob.glob(f'{experiment.path}/*')
levels

In [ ]:
def read_firestarter_from_lic(folder: str) -> pd.DataFrame:
    """
    folder: The folder that contains the measurements for the different number of cores.
    """
    df = pd.DataFrame()

    for file in glob.glob(f'{folder}/firestarter-*'):
        _name, index = Path(file).stem.split('-')
        index = int(index)

        read_df = pd.read_csv(file)

        for _index, row in read_df.iterrows():
            metric = row['metric']
            value = row['average']
            df = pd.concat([df, pd.DataFrame([{'metric': metric, 'value': value, 'index': index}])])

    return df

In [ ]:
def read_uncore_from_lic(folder: str) -> pd.DataFrame:
    """
    folder: The folder that contains the measurements for the different number of cores.
    """
    df = pd.DataFrame()

    for file in glob.glob(f'{folder}/uncore-freq-*'):
        _name, _name2, index = Path(file).stem.split('-')
        index = int(index)

        read_df = pd.read_csv(file)

        for _index, row in read_df.iterrows():
            metric = row['name']
            value = row['average']
            df = pd.concat([df, pd.DataFrame([{'metric': metric, 'value': value, 'index': index}])])

    return df

In [ ]:
all_data = pd.DataFrame()

for level in levels:
    lic_data = read_firestarter_from_lic(level)
    lic_data['level'] = os.path.basename(level)
    all_data = pd.concat([all_data, lic_data])

    lic_data = read_uncore_from_lic(level)
    lic_data['level'] = os.path.basename(level)
    all_data = pd.concat([all_data, lic_data])

all_data

In [ ]:
license_level_to_name = {
    'lic0': 'SSE',
    'lic1': 'AVX2',
    'lic2': 'AVX512',
    'lic3': 'AMX'
}

PERF_IPC = 'IPC of the compute kernel'
MAX_TURBO_FREQ = 'Maximum opportunistic turbo frequency [GHz]'
POWER_WATT = 'Power draw of package 0 [W]'
UNCORE_FREQ = 'Uncore frequency of package 0 [GHz]'

metric_name_map = {
    'perf-ipc': PERF_IPC,
    'perf-freq': MAX_TURBO_FREQ,
    'ipc-estimate': 'ipc-estimate',
    'package_00_die_00': UNCORE_FREQ,
    'package_01_die_00': UNCORE_FREQ,
    'sysfs-powercap-rapl': 'sysfs-powercap-rapl',
    'sysfs-powercap-rapl/package-0': POWER_WATT,
    'sysfs-powercap-rapl/package-0-dram': 'sysfs-powercap-rapl/package-0-dram',
    'sysfs-powercap-rapl/package-1': 'sysfs-powercap-rapl/package-1',
    'sysfs-powercap-rapl/package-1-dram': 'sysfs-powercap-rapl/package-1-dram',
}

all_data_copy = all_data.copy()

all_data_copy['level'] = all_data_copy['level'].apply(lambda v: license_level_to_name[v])
all_data_copy['metric'] = all_data_copy['metric'].apply(lambda v: metric_name_map[v])

all_data_pivot = all_data_copy.pivot_table('value', ['index', 'level'], 'metric')

all_data_pivot[UNCORE_FREQ] = all_data_pivot[UNCORE_FREQ] / 1e6

all_data_pivot

In [ ]:
PERFORMANCE = 'Performance [G Instructions/s]'
PERFORMANCE_DEVIATION = 'Deviation from mean performance [%]'
ENERGY_EFFICIENCY = 'Energy Efficiency [G Instructions/Watt/s]'
ENERGY_EFFICIENCY_DEVIATION = 'Deviation from maximum energy efficiency [%]'

all_data_pivot[PERFORMANCE] = all_data_pivot[PERF_IPC] * all_data_pivot[MAX_TURBO_FREQ]
all_data_pivot[ENERGY_EFFICIENCY] = all_data_pivot[PERFORMANCE] / all_data_pivot[POWER_WATT]

In [ ]:
df_performance_deviation = all_data_pivot[PERFORMANCE].reset_index()

for key, value in df_performance_deviation.groupby("level"):
    mean = value[PERFORMANCE].mean()
    df_performance_deviation.loc[value.index, [PERFORMANCE_DEVIATION]] = (value[PERFORMANCE] / mean - 1) * 100

print(df_performance_deviation)

In [ ]:
hue_order=['AMX', 'AVX512', 'AVX2', 'SSE']

In [ ]:
ax = sns.ecdfplot(data=all_data_pivot[POWER_WATT].reset_index(), x=POWER_WATT, hue='level', hue_order=hue_order)
Plotting.savefig(experiment, 'package-power-ecdf.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
ax = sns.ecdfplot(data=all_data_pivot[PERF_IPC].reset_index(), x=PERF_IPC, hue='level', hue_order=hue_order)
Plotting.savefig(experiment, 'perf-ipc-ecdf.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
ax = sns.ecdfplot(data=all_data_pivot[MAX_TURBO_FREQ].reset_index(), x=MAX_TURBO_FREQ, hue='level', hue_order=hue_order)
Plotting.savefig(experiment, 'perf-freq-ecdf.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
ax = sns.ecdfplot(data=all_data_pivot[UNCORE_FREQ].reset_index(), x=UNCORE_FREQ, hue='level', hue_order=hue_order)
Plotting.savefig(experiment, 'uncore-freq-ecdf.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
df_performance_deviation = all_data_pivot[PERFORMANCE].reset_index()

for key, value in df_performance_deviation.groupby("level"):
    mean = value[PERFORMANCE].mean()
    df_performance_deviation.loc[value.index, [PERFORMANCE_DEVIATION]] = (value[PERFORMANCE] / mean - 1) * 100

print(df_performance_deviation)

In [ ]:
ax = sns.ecdfplot(data=df_performance_deviation, x=PERFORMANCE_DEVIATION, hue='level', hue_order=hue_order)
Plotting.savefig(experiment, 'performance-ecdf.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
df_energy_efficiency_deviation = all_data_pivot.reset_index()

for key, value in df_energy_efficiency_deviation.groupby("level"):
    max = value[ENERGY_EFFICIENCY].max()
    df_energy_efficiency_deviation.loc[value.index, [ENERGY_EFFICIENCY_DEVIATION]] = (value[ENERGY_EFFICIENCY] / max - 1)  * 100

print(df_energy_efficiency_deviation)

In [ ]:
ax = sns.ecdfplot(data=df_energy_efficiency_deviation, x=ENERGY_EFFICIENCY_DEVIATION, hue='level', hue_order=hue_order)
Plotting.savefig(experiment, 'energy-efficiency-ecdf.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)